# Applied social network analysis in Python
----

**Applied social network analysis in Python** was the last course in the **Applied Data Science with Python** specialization. This notebook shows my work for the four assignments that I had to solve during this course:

* **Assignment 1: Creating and manipulating graphs**

In this assignment, I've acquired experience in creating bipartite graphs, how to add node attributes and how to find a weighted projection of the graph.

* **Assignment 2:  Network Connectivity**

For this assignment I've analyzed the internal email communication network between employees of a mid-sized manufacturing company. 

* **Assignment 3:  Influence Measures and Network Centralization**

In this part of the assignment I've computed various measures of centrality on two networks: *a friendship network in Part 1*, and a *blog network in Part 2*.

* **Assignment 4: Network Evolution**

This assignment was split in two parts. In the first part of the assignment I've analyzed randomly generated graphs and determined which algorithm created them. For the second part of this assignment I've worked with a company's email network to predict salaries and new connections.

## Assignment 1 
----

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [7]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [8]:
def answer_one():
    # Your Code Here
    df = pd.read_table('datasets/Employee_Movie_Choices.txt')
    G = nx.Graph()
    G.add_nodes_from(df['#Employee'].values, bipartite=0)
    G.add_nodes_from(df['Movie'].values, bipartite=1)
    for idx, row in df.iterrows():
        G.add_edge(row['#Employee'],row['Movie'])
    return G

answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [12]:
def answer_two():
    # Your Code Here
    df = pd.read_table('datasets/Employee_Movie_Choices.txt')
    G = nx.Graph()
    G.add_nodes_from(df['#Employee'].values, bipartite=0, role='employee')
    G.add_nodes_from(df['Movie'].values, bipartite=1, role='movie')
    for idx, row in df.iterrows():
        G.add_edge(row['#Employee'],row['Movie'])  
    return G

answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [15]:
def answer_three():
    # Your Code Here
    df = pd.read_table('datasets/Employee_Movie_Choices.txt')
    X = set(df['#Employee'].values)
    G = answer_two()
    P = bipartite.weighted_projected_graph(G, X)
    return P

answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [18]:
def answer_four(): 
    # Your Code Here
    df = pd.read_table('datasets/Employee_Relationships.txt',
                       header=None, names=['first_empl', 'sec_empl', 'rel'])
    P = answer_three()
    num_movies = []
    df['num_movies']  = 0
    for idx, row in df.iterrows():
        if (row['first_empl'], row['sec_empl']) in P.edges():
            df.loc[idx, 'num_movies'] = P.edge[row['first_empl']][row['sec_empl']]['weight']
        elif (row['sec_empl'], row['first_empl']) in P.edges():
             df.loc[idx, 'num_movies'] = P.edge[row['sec_empl']][row['first_empl']]['weight']
        else:
            df.loc[idx, 'num_movies'] = 0
    return float(df['rel'].corr(df['num_movies']))

answer_four()

## Assignment 2
----

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one(): 
    # Your Code Here
    G = nx.read_edgelist('datasets/email_network.txt', create_using=nx.MultiDiGraph(), data=(('weight',float),))
    return G

answer_one()

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [4]:
def answer_two():
    # Your Code Here
    G = answer_one()
    return (len(G.nodes()), len(G.edges()))

answer_two()

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [5]:
def answer_three():
    # Your Code Here
    G = answer_one()
    return (nx.is_strongly_connected(G), nx.is_weakly_connected(G))

answer_three()

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [6]:
def answer_four():
    # Your Code Here
    G = answer_one()
    largest = max(nx.weakly_connected_component_subgraphs(G), key=len)
    return len(largest.nodes())

answer_four()

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [7]:
def answer_five():
    # Your Code Here
    G = answer_one()
    largest = max(nx.strongly_connected_component_subgraphs(G), key=len)
    return len(largest.nodes())

answer_five()

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [8]:
def answer_six():
    # Your Code Here
    G = answer_one()
    G_sc = max(nx.strongly_connected_component_subgraphs(G), key=len) 
    return G_sc

answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [9]:
def answer_seven():
    # Your Code Here
    G_sc = answer_six()
    return nx.average_shortest_path_length(G_sc)

answer_seven()

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [10]:
def answer_eight():
    # Your Code Here
    G_sc = answer_six()
    return nx.diameter(G_sc)

answer_eight()

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [11]:
def answer_nine():
    # Your Code Here
    G_sc = answer_six()
    return set(nx.periphery(G_sc))

answer_nine()

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [12]:
def answer_ten():
    # Your Code Here
    G_sc = answer_six()
    return set(nx.center(G_sc))

answer_ten()

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [13]:
def answer_eleven():
    # Your Code Here
    G_sc = answer_six()
    diameter = nx.diameter(G_sc)
    paths = nx.shortest_path_length(G_sc)
    max_node, max_num = 0, 0
    for node in paths.keys():
        satis_conn = len([x for x in paths[node].keys() if paths[node][x]==diameter])
        if satis_conn>max_num:
            max_node, max_num = node, satis_conn   
    return (max_node, max_num)

answer_eleven()

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [14]:
def answer_twelve():
    # Your Code Here
    G_sc = answer_six()
    return int(nx.node_connectivity(G_sc, '38', '97')-1)

answer_twelve()

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [15]:
def answer_thirteen():
    # Your Code Here
    G_sc = answer_six()
    G_un = G_sc.to_undirected()
    return nx.Graph(G_un)

answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [16]:
def answer_fourteen():
    # Your Code Here
    G_un = answer_thirteen()
    return (nx.transitivity(G_un), nx.average_clustering(G_un))

answer_fourteen()

## Assignment 3
----

In this assignment you will explore measures of centrality on two networks, a friendship network in Part 1, and a blog network in Part 2.

## Part 1

Answer questions 1-4 using the network `G1`, a network of friendships at a university department. Each node corresponds to a person, and an edge indicates friendship. 

*The network has been loaded as networkx graph object `G1`.*

In [1]:
import networkx as nx

G1 = nx.read_gml('datasets/friendships.gml')

### Question 1

Find the degree centrality, closeness centrality, and normalized betweeness centrality (excluding endpoints) of node 100.

*This function should return a tuple of floats `(degree_centrality, closeness_centrality, betweenness_centrality)`.*

In [19]:
def answer_one():
    # Your Code Here
    centrality = nx.degree_centrality(G1)[100]
    closeness = nx.closeness_centrality(G1)[100]
    betweenness = nx.betweenness_centrality(G1)[100] 
    return (centrality, closeness, betweenness)

answer_one()

#### For Questions 2, 3, and 4, assume that you do not know anything about the structure of the network, except for the all the centrality values of the nodes. That is, use one of the covered centrality measures to rank the nodes and find the most appropriate candidate.

### Question 2

Suppose you are employed by an online shopping website and are tasked with selecting one user in network G1 to send an online shopping voucher to. We expect that the user who receives the voucher will send it to their friends in the network.  You want the voucher to reach as many nodes as possible. The voucher can be forwarded to multiple users at the same time, but the travel distance of the voucher is limited to one step, which means if the voucher travels more than one step in this network, it is no longer valid. Apply your knowledge in network centrality to select the best candidate for the voucher. 

*This function should return an integer, the name of the node.*

In [21]:
def answer_two():    
    # Your Code Here
    centrality = nx.degree_centrality(G1)
    return max(centrality.keys(), key=(lambda key: centrality[key]))

answer_two()

### Question 3

Now the limit of the voucher’s travel distance has been removed. Because the network is connected, regardless of who you pick, every node in the network will eventually receive the voucher. However, we now want to ensure that the voucher reaches the nodes in the lowest average number of hops.

How would you change your selection strategy? Write a function to tell us who is the best candidate in the network under this condition.

*This function should return an integer, the name of the node.*

In [23]:
def answer_three():  
    # Your Code Here
    closeness = nx.closeness_centrality(G1)
    return max(closeness.keys(), key=(lambda key: closeness[key]))

answer_three()

### Question 4

Assume the restriction on the voucher’s travel distance is still removed, but now a competitor has developed a strategy to remove a person from the network in order to disrupt the distribution of your company’s voucher. Your competitor is specifically targeting people who are often bridges of information flow between other pairs of people. Identify the single riskiest person to be removed under your competitor’s strategy?

*This function should return an integer, the name of the node.*

In [25]:
def answer_four():
    # Your Code Here
    betweenness = nx.betweenness_centrality(G1)
    return max(betweenness.keys(), key=(lambda key: betweenness[key]))

answer_four()

## Part 2

`G2` is a directed network of political blogs, where nodes correspond to a blog and edges correspond to links between blogs. Use your knowledge of PageRank and HITS to answer Questions 5-9.

In [27]:
G2 = nx.read_gml('datasets/blogs.gml')

### Question 5

Apply the Scaled Page Rank Algorithm to this network. Find the Page Rank of node 'realclearpolitics.com' with damping value 0.85.

*This function should return a float.*

In [28]:
def answer_five():
    # Your Code Here
    return nx.pagerank(G2, alpha=0.85)['realclearpolitics.com']

answer_five()

### Question 6

Apply the Scaled Page Rank Algorithm to this network with damping value 0.85. Find the 5 nodes with highest Page Rank. 

*This function should return a list of the top 5 blogs in desending order of Page Rank.*

In [30]:
def answer_six():    
    # Your Code Here
    import numpy as np
    pageranks = nx.pagerank(G2, alpha=0.85)
    pages = np.array(list(pageranks.keys()))
    ranks = np.array(list(pageranks.values()))
    return list(pages[ranks.argsort()[-5:][::-1]])

answer_six()

### Question 7

Apply the HITS Algorithm to the network to find the hub and authority scores of node 'realclearpolitics.com'. 

*Your result should return a tuple of floats `(hub_score, authority_score)`.*

In [34]:
def answer_seven():
    # Your Code Here
    hubs, authorities = nx.hits(G2)
    return (hubs['realclearpolitics.com'], authorities['realclearpolitics.com'])

answer_seven()

### Question 8 

Apply the HITS Algorithm to this network to find the 5 nodes with highest hub scores.

*This function should return a list of the top 5 blogs in desending order of hub scores.*

In [36]:
def answer_eight():
    # Your Code Here
    import numpy as np
    hubs, authorities = nx.hits(G2)
    blogs = np.array(list(hubs.keys()))
    hubs = np.array(list(hubs.values()))
    return list(blogs[hubs.argsort()[-5:][::-1]])

answer_eight()

### Question 9 

Apply the HITS Algorithm to this network to find the 5 nodes with highest authority scores.

*This function should return a list of the top 5 blogs in desending order of authority scores.*

In [38]:
def answer_nine():
    # Your Code Here
    import numpy as np
    hubs, auth = nx.hits(G2)
    blogs = np.array(list(auth.keys()))
    auth = np.array(list(auth.values()))
    return list(blogs[auth.argsort()[-5:][::-1]])

answer_nine()

# Assignment 4
----

In [40]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [41]:
P1_Graphs = pickle.load(open('datasets/A4_graphs','rb'))
P1_Graphs

`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [42]:
def graph_identification():
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [44]:
G = nx.read_gpickle('datasets/email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [45]:
def salary_predictions():
    # Your Code Here
    from sklearn.ensemble import GradientBoostingClassifier
    input_data = pd.DataFrame()
    target = nx.get_node_attributes(G, 'ManagementSalary')
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    for node in target.keys():
        row_features = pd.DataFrame([[degree_centrality[node], closeness_centrality[node],
                                      betweenness_centrality[node], target[node]]], index=[node])
        input_data = pd.concat([input_data, row_features], axis=0)
    train_data = input_data[~input_data[3].isnull()]
    test_data = input_data[input_data[3].isnull()]
    clf = GradientBoostingClassifier()
    clf.fit(train_data[[0,1,2]].values, train_data[3].values)
    preds = clf.predict_proba(test_data[[0,1,2]].values)[:,1]  
    return pd.Series(preds, index=test_data.index)

salary_predictions().head(10)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [46]:
future_connections = pd.read_csv('datasets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [47]:
def new_connections_predictions():  
     # Your Code Here
    from sklearn.ensemble import GradientBoostingClassifier
    
    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]
    train_data = future_connections[~future_connections['Future Connection'].isnull()]
    test_data = future_connections[future_connections['Future Connection'].isnull()]
    clf = GradientBoostingClassifier()
    clf.fit(train_data[['pref_attachment','comm_neighbors']].values, train_data['Future Connection'].values)
    preds = clf.predict_proba(test_data[['pref_attachment','comm_neighbors']].values)[:,1]
    return pd.Series(preds, index=test_data.index)

new_connections_predictions().head(10)